In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
! pip install open3d
! pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
import open3d as o3d
import numpy as np

In [2]:
%cd drive/MyDrive/EPTnet/point_transformer_lib

/content/drive/MyDrive/EPTnet/point_transformer_lib


In [3]:
! python setup.py build_ext install

running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:415: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:425: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDepr

In [5]:
%cd ..

/content/drive/MyDrive/EPTnet


In [18]:
! python -m tool.test --config config/IntrA/IntrA_pointtransformer_seg_repro.yaml sample_points 512

[2024-05-18 00:30:09,668 INFO test.py line 57 15325] arch: IntrA_pointtransformer_seg_repro
base_lr: 0.001
batch_size_train: 8
batch_size_val: 8
classes: 2
data_name: IntrA
data_root: data/IntrA/
downsampling_ratio: 0.5
drop_rate: 0.5
epochs: 400
eval_freq: 1
evaluate: True
fea_dim: 6
folds: [0, 1, 2, 3, 4]
loop: 1
manual_seed: 666
momentum: 0.9
n_neighbors: 8
ngpus_per_node: 1
npoints: 16
num_edge_neighbor: 4
num_votes: 3
num_workers: 8
print_freq: 1
resume: None
sample_points: 512
save_freq: 1
save_path: None
start_epoch: 0
sync_bn: False
temp: 1.0
test_points: 512
test_times: 10
train_gpu: [0]
use_normals: True
use_uniform_sample: True
use_xyz: True
weight: None
weight_contra: 0.2
weight_decay: 0.0
weight_edge: 1
weight_refine: 1
[2024-05-18 00:30:09,669 INFO test.py line 94 15325] ===============Test fold 0===============
[2024-05-18 00:30:09,669 INFO test.py line 97 15325] => Creating model ...
[2024-05-18 00:30:10,084 INFO test.py line 119 15325] => Loading data ...
[2024-05-18 0

In [ ]:
! python -m tool.train --config config/IntrA/IntrA_pointtransformer_seg_r=oepro.yaml sample_points 512

[2024-05-18 01:01:32,210 INFO train.py line 61 23789] arch: IntrA_pointtransformer_seg_repro
base_lr: 0.001
batch_size_train: 8
batch_size_val: 8
classes: 2
data_name: IntrA
data_root: data/IntrA/
downsampling_ratio: 0.5
drop_rate: 0.5
epochs: 400
eval_freq: 1
evaluate: True
fea_dim: 6
folds: [0, 1, 2, 3, 4]
loop: 1
manual_seed: 666
momentum: 0.9
n_neighbors: 8
ngpus_per_node: 1
npoints: 16
num_edge_neighbor: 4
num_votes: 3
num_workers: 8
print_freq: 1
resume: None
sample_points: 512
save_freq: 1
save_path: /content/drive/MyDrive/EPTnet/exp/IntrA/pointtransformer_seg_repro_test
start_epoch: 0
sync_bn: False
temp: 1.0
test_points: 512
test_times: 10
train_gpu: [0]
use_normals: True
use_uniform_sample: True
use_xyz: True
weight: None
weight_contra: 0.2
weight_decay: 0.0
weight_edge: 1
weight_refine: 1
[2024-05-18 01:01:32,553 INFO train.py line 79 23789] ===============test fold 0===============
[2024-05-18 01:01:32,554 INFO train.py line 81 23789] =====================> Creating model .

### Visualization

In [ ]:
from models.point_transformer_seg import PointTransformerSemSegmentation as Model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
testPath = "data/IntrA/"
testFiles = os.listdir(testPath)

model = Model()
weightPath = "PointNet.pth"
model.load_state_dict(torch.load(weightPath, map_location=device))

for testFile in testFiles:
    pcdPath = os.path.join(testPath, testFile)
    pcd = o3d.io.read_point_cloud(pcdPath, format='xyz')
    points = np.asarray(pcd.points)
    pointNP = points.copy()

    mu = np.mean(points, axis=0)
    var = np.mean(np.square(points-mu))
    points = (points-mu) / np.sqrt(var)
    points = torch.tensor(points, dtype=torch.float32).transpose(1, 0)
    points = points.view(1, points.shape[0], points.shape[1])


    model.eval()
    with torch.no_grad():
        output = torch.squeeze(model(points.to(device)))
        predict = torch.softmax(output, dim=0).cpu()
        predict_cla = torch.argmax(predict, dim=0).numpy()

        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(pointNP)
        colors = np.array([color_map[label] for label in predict_cla])
        point_cloud.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([point_cloud])